# Answer Questions

Query the index and feed the top N results to ChatGPT

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [ ]:
import os

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Pinecone
import pinecone

In [ ]:
text_generation_model_name = 'gpt-3.5-turbo'
text_embedding_model_name = "text-embedding-ada-002"
index_name = "conf-ada-002"
text_field = "text"

## Initialize OpenAI and Pinecone

In [ ]:
embedder = OpenAIEmbeddings(
    model=text_embedding_model_name,
    openai_api_key=os.environ['OPENAI_API_KEY'],
)

In [ ]:
pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'], 
    environment=os.environ['PINECONE_ENV'],
)
index = pinecone.Index(index_name)
index.describe_index_stats()

In [ ]:
vectorstore = Pinecone(
    index, embedder, text_field
)

## Answer a question

In [ ]:
llm = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    model_name=text_generation_model_name,
    temperature=0.0
)

In [ ]:
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum. Keep the answer as concise as possible. 
Always say "thanks for asking!" at the end |of the answer. 
{summaries}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={'k': 20}),
    return_source_documents=True,
    reduce_k_below_max_tokens=True,
    max_tokens_limit=2000,
    # TODO customize prompt
    # chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    verbose=True,
)

In [ ]:
query = "What are the blessings of keeping the sabbath day holy?"

qa_chain({"question": query})